In [1]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from math import sqrt
import warnings
import gc
warnings.filterwarnings('ignore')

In [2]:
path_train = "../resource/PINGAN-2018-train_demo.csv"
path_test = "../resource/PINGAN-2018-test_demo.csv"

all_feature_list = ['CALLSTATE', 'DIRECTION', 'HEIGHT', 'LATITUDE', 'LONGITUDE', 'SPEED',
       'TERMINALNO', 'TIME', 'TRIP_ID', 'Y', 'time', 'date', 'hour', 'minute',
       'trip_max', 'lon_max', 'lon_min', 'lon', 'lat_max', 'lat_min', 'lat',
       'heg_max', 'heg_min', 'heg_mean', 'heg', 'vol', 'sp_max', 'sp_mean',
       'call0', 'call1', 'call_ratio_0', 'call_ratio_1','dis']

use_feature_list = [
       'trip_max', 'lon_max', 'lon_min', 'lon', 'lat_max', 'lat_min', 'lat',
       'heg_max', 'heg_min', 'heg_mean', 'heg', 'vol', 'sp_max', 'sp_mean',
       'dis', 'ave_dri_time', 'dri_time']

In [3]:
def load_data(path_train,path_test):
    train_data = pd.read_csv(path_train)
    test_data = pd.read_csv(path_test)
    return train_data,test_data

In [4]:
train_data,test_data = load_data(path_train,path_test)
print(train_data.shape,test_data.shape)
# 去重
train_data.drop_duplicates(subset=['TERMINALNO','TIME'],inplace=True)
test_data.drop_duplicates(subset=['TERMINALNO', 'TIME'], inplace=True)
train_Y = train_data[['TERMINALNO','Y']].drop_duplicates().reset_index(drop=True)
# print(train_Y)
# 拼接训练集和测试集进行特征工程
TRAIN_ID_MAX = train_data['TERMINALNO'].max() + 10
test_data['TERMINALNO'] = test_data['TERMINALNO'] + TRAIN_ID_MAX
data = pd.concat([train_data, test_data])
# print(" drop duplicates")
# 重置index
data.reset_index(drop=True, inplace=True)
del train_data, test_data
gc.collect()

(69306, 10) (69306, 9)


59

In [5]:
#时间处理
def time_datetime(value):
    format = '%Y%m%d%H%M'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

def time_date(value):
    format = '%Y%m%d'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

def time_hour(value):
    format = '%H'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)
def time_minute(value):
    format = '%M'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

#转换成时刻
data['time']=data['TIME'].apply(time_datetime)
data['date']=data['TIME'].apply(time_date)
data['hour']=data['TIME'].apply(time_hour)
data['minute']=data['TIME'].apply(time_minute)

In [6]:
# trip_max
feature = pd.DataFrame()
feature[['TERMINALNO', 'trip_max']] = pd.DataFrame(data['TRIP_ID'].groupby(data['TERMINALNO']).max()).reset_index()[
    ['TERMINALNO', 'TRIP_ID']]

In [7]:
# lon_max lon_min lon
# lonmax = pd.DataFrame()
# lonmin = pd.DataFrame()
feature[['TERMINALNO', 'lon_max']] = pd.DataFrame(data['LONGITUDE'].groupby(data['TERMINALNO']).max()).reset_index()[
    ['TERMINALNO', 'LONGITUDE']]
feature[['TERMINALNO', 'lon_min']] = pd.DataFrame(data['LONGITUDE'].groupby(data['TERMINALNO']).min()).reset_index()[
    ['TERMINALNO', 'LONGITUDE']]
# feature = pd.merge(feature, lonmax, how='left', on='TERMINALNO')
# feature = pd.merge(feature, lonmin, how='left', on='TERMINALNO')
feature['lon'] = feature['lon_max'] - feature['lon_min']
# del lonmax,lonmin
# gc.collect()

In [8]:
# lat_max lat_min lat
# latmax = pd.DataFrame()
# latmin = pd.DataFrame()
feature[['TERMINALNO', 'lat_max']] = pd.DataFrame(data['LATITUDE'].groupby(data['TERMINALNO']).max()).reset_index()[
    ['TERMINALNO', 'LATITUDE']]
feature[['TERMINALNO', 'lat_min']] = pd.DataFrame(data['LATITUDE'].groupby(data['TERMINALNO']).min()).reset_index()[
    ['TERMINALNO', 'LATITUDE']]
# feature = pd.merge(feature, latmax, how='left', on='TERMINALNO')
# feature = pd.merge(feature, latmin, how='left', on='TERMINALNO')
feature['lat'] = feature['lat_max'] - feature['lat_min']
# del latmax,latmin
# gc.collect()

In [9]:
# heg_max heg_min heg_mean heg
# hegmax = pd.DataFrame()
# hegmin = pd.DataFrame()
# hegmean = pd.DataFrame()
feature[['TERMINALNO', 'heg_max']] = pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).max()).reset_index()[
    ['TERMINALNO', 'HEIGHT']]
feature[['TERMINALNO', 'heg_min']] = pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).min()).reset_index()[
    ['TERMINALNO', 'HEIGHT']]
feature[['TERMINALNO', 'heg_mean']] = pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).mean()).reset_index()[
    ['TERMINALNO', 'HEIGHT']]
# feature = pd.merge(feature, hegmax, how='left', on='TERMINALNO')
# feature = pd.merge(feature, hegmin, how='left', on='TERMINALNO')
# feature = pd.merge(feature, hegmean, how='left', on='TERMINALNO')
feature['heg'] = feature['heg_max'] - feature['heg_min']
# del hegmax,hegmean,hegmin
# gc.collect()

In [10]:
# volu 活动区间体积
feature['vol'] = feature['lon'] * feature['lat'] * feature['heg']
print("lon,lat,heg precessed....")

lon,lat,heg precessed....


In [11]:
# 速度 sp_max sp_mean
# spmax = pd.DataFrame()
# spmean = pd.DataFrame()
feature[['TERMINALNO', 'sp_max']] = pd.DataFrame(data['SPEED'].groupby(data['TERMINALNO']).max()).reset_index()[
    ['TERMINALNO', 'SPEED']]
feature[['TERMINALNO', 'sp_mean']] = pd.DataFrame(data['SPEED'].groupby(data['TERMINALNO']).mean()).reset_index()[
    ['TERMINALNO', 'SPEED']]
# feature = pd.merge(feature, spmax, how='left', on='TERMINALNO')
# feature = pd.merge(feature, spmean, how='left', on='TERMINALNO')
print("sp_max,sp_mean finished")
# del spmax,spmean
# gc.collect()

sp_max,sp_mean finished


In [12]:
# # callstate
# # call0 = pd.DataFrame()
# # call1 = pd.DataFrame()
# feature[['TERMINALNO', 'call0']] = \
# pd.DataFrame(data['CALLSTATE'][data['CALLSTATE'] == 0].groupby(data['TERMINALNO']).count()).reset_index()[
#     ['TERMINALNO', 'CALLSTATE']]
# feature[['TERMINALNO', 'call1']] = \
# pd.DataFrame(data['CALLSTATE'][data['CALLSTATE'] > 0].groupby(data['TERMINALNO']).count()).reset_index()[
#     ['TERMINALNO', 'CALLSTATE']]
# # feature = pd.merge(feature, call0, how='left', on='TERMINALNO')
# # feature = pd.merge(feature, call1, how='left', on='TERMINALNO')

# feature['call0'].fillna(0, inplace=True)
# feature['call1'].fillna(0, inplace=True)
# feature['call_ratio_0'] = feature['call0'] / (feature['call0'] + feature['call1'])
# feature['call_ratio_1'] = feature['call1'] / (feature['call0'] + feature['call1'])
# # del call0,call1
# # gc.collect()
# print("call state finished")

In [ ]:
feature.info()

In [13]:
# 行程
# 对每个 USER 按 TIME 排序
sortdata = data.sort_values(['TERMINALNO', 'time']).reset_index(drop=True)
# 删除TRIP_ID后去重
# del sortdata['TRIP_ID']
# gc.collect()
# sortdata.drop_duplicates(inplace=True)
# 计算经纬度差
sortdata['difflat'] = sortdata.groupby(['TERMINALNO'])['LATITUDE'].diff()
sortdata['difflon'] = sortdata.groupby(['TERMINALNO'])['LONGITUDE'].diff()
# 对每个用户的第一个经纬度差置0
sortdata.fillna(0.0, inplace=True)
# 计算单个距离
sortdata['dis2'] = sortdata['difflat'] ** 2 + sortdata['difflon'] ** 2
sortdata['dis'] = sortdata['dis2'].apply(sqrt)
del sortdata['dis2']
gc.collect()
# 计算总行程
# disdata = pd.DataFrame()
# disdata[['TERMINALNO','dis']]=sortdata['dis'].groupby(['TERMINALNO']).sum()
disdata = sortdata['dis'].groupby(sortdata['TERMINALNO']).sum().reset_index()
feature = pd.merge(feature, disdata, how='left', on='TERMINALNO')
del sortdata,disdata
gc.collect()
print("dis finished")

# 驾驶时长
# 1.去重
dri_time = data[['TERMINALNO', 'TIME', 'TRIP_ID']]
dri_time.drop_duplicates(subset=['TERMINALNO', 'TIME'], inplace=True)
# 2.按 TERMINALNO 和 time 排序
dri_time.sort_values(['TERMINALNO', 'TIME'], inplace=True)
dri_time['diff_time'] = dri_time.groupby(['TERMINALNO'])['TIME'].diff()
dri_time.fillna(0.0, inplace=True)
# 3.时间换算
dri_time['diff_time'] = dri_time['diff_time'].apply(lambda x: x / 60)
# 4.如果时间间隔大于20分钟则按新行程处理，置0
def f(x):
    if x >= 20:
        return 0
    else:
        return x
dri_time['diff_time'] = dri_time['diff_time'].apply(f)
# 5.计算驾驶总时长
dri_t = pd.DataFrame()
dri_t[['TERMINALNO', 'dri_time']] = dri_time['diff_time'].groupby(dri_time['TERMINALNO']).sum().reset_index()[
    ['TERMINALNO', 'diff_time']]
feature = pd.merge(feature, dri_t, how='left', on='TERMINALNO')
# 6.平均时长
feature['ave_dri_time'] = feature['dri_time'] / feature['trip_max']
del dri_t, dri_time
gc.collect()
# use_feature_list = [
#     'trip_max', 'lon_max', 'lon_min', 'lon', 'lat_max', 'lat_min', 'lat',
#     'heg_max', 'heg_min', 'heg_mean', 'heg', 'vol', 'sp_max', 'sp_mean',
#     'call_ratio_0', 'call_ratio_1', 'dis', 'ave_dri_time', 'dri_time']
print("feature end...")

dis finished
feature end...


In [14]:
# 归一化
feature['trip_max'] = feature['trip_max'].apply(
    lambda x: (x - feature['trip_max'].min()) / (feature['trip_max'].max() - feature['trip_max'].min()))
feature['lon_max'] = feature['lon_max'].apply(
    lambda x: (x - feature['lon_max'].min()) / (feature['lon_max'].max() - feature['lon_max'].min()))
feature['lon_min'] = feature['lon_min'].apply(
    lambda x: (x - feature['lon_min'].min()) / (feature['lon_min'].max() - feature['lon_min'].min()))
feature['lon'] = feature['lon'].apply(
    lambda x: (x - feature['lon'].min()) / (feature['lon'].max() - feature['lon'].min()))
feature['lat_min'] = feature['lat_min'].apply(
    lambda x: (x - feature['lat_min'].min()) / (feature['lat_min'].max() - feature['lat_min'].min()))
feature['lat_max'] = feature['lat_max'].apply(
    lambda x: (x - feature['lat_max'].min()) / (feature['lat_max'].max() - feature['lat_max'].min()))
feature['lat'] = feature['lat'].apply(
    lambda x: (x - feature['lat'].min()) / (feature['lat'].max() - feature['lat'].min()))
feature['heg_min'] = feature['heg_min'].apply(
    lambda x: (x - feature['heg_min'].min()) / (feature['heg_min'].max() - feature['heg_min'].min()))
feature['heg_max'] = feature['heg_max'].apply(
    lambda x: (x - feature['heg_max'].min()) / (feature['heg_max'].max() - feature['heg_max'].min()))
feature['heg'] = feature['heg'].apply(
    lambda x: (x - feature['heg'].min()) / (feature['heg'].max() - feature['heg'].min()))
feature['heg_mean'] = feature['heg_mean'].apply(
    lambda x: (x - feature['heg_mean'].min()) / (feature['heg_mean'].max() - feature['heg_mean'].min()))
feature['vol'] = feature['vol'].apply(
    lambda x: (x - feature['vol'].min()) / (feature['vol'].max() - feature['vol'].min()))
feature['sp_max'] = feature['sp_max'].apply(
    lambda x: (x - feature['sp_max'].min()) / (feature['sp_max'].max() - feature['sp_max'].min()))
feature['sp_mean'] = feature['sp_mean'].apply(
    lambda x: (x - feature['sp_mean'].min()) / (feature['sp_mean'].max() - feature['sp_mean'].min()))
feature['ave_dri_time'] = feature['ave_dri_time'].apply(
    lambda x: (x - feature['ave_dri_time'].min()) / (feature['ave_dri_time'].max() - feature['ave_dri_time'].min()))
feature['dri_time'] = feature['dri_time'].apply(
    lambda x: (x - feature['dri_time'].min()) / (feature['dri_time'].max() - feature['dri_time'].min()))
feature['dis'] = feature['dis'].apply(
    lambda x: (x - feature['dis'].min()) / (feature['dis'].max() - feature['dis'].min()))
print("data normalization..")
print("Feature End. feature shape:" + str(feature.shape))
print("generate train & test set")

data normalization..
Feature End. feature shape:(200, 18)
generate train & test set


In [15]:
# 切割训练集和测试集
# train = data[0:TRAIN_LENGTH]
# test = data[TRAIN_LENGTH:]
feature = pd.merge(feature, train_Y, how='left', on='TERMINALNO')
train = feature[0:len(train_Y)]
test = feature[len(train_Y):]
# test = pd.merge(test, feature, how='left', on='TERMINALNO')
train['Y'] = train['Y'].apply(lambda x: ((x - train['Y'].min()) / (train['Y'].max() - train['Y'].min())))
# 训练集和验证集划分
train_train, train_val = train_test_split(train, test_size=0.2, random_state=42)
print("train_train_shape:"+str(train_train.shape)+"  train_val_shape:"+str(train_val.shape))

train_train_shape:(80, 19)  train_val_shape:(20, 19)


In [18]:
test

,TERMINALNO,trip_max,lon_max,lon_min,lon,lat_max,lat_min,lat,heg_max,heg_min,heg_mean,heg,vol,sp_max,sp_mean,dis,dri_time,ave_dri_time,Y
100,311,0.198198,0.936877,0.947739,0.256717,0.976947,0.879034,0.180038,0.064576,0.188562,0.016106,0.093868,0.039362,0.393851,0.493323,2.388804e-01,0.118051,0.012199,NaN
101,312,0.292793,0.748756,0.720134,0.535277,0.383671,0.060535,0.595878,0.153262,0.113481,0.024443,0.261162,0.720268,0.491654,0.186809,7.993322e-01,0.455965,0.041473,NaN
102,313,0.162162,0.671824,0.709979,0.137334,0.293424,0.040572,0.482068,0.035725,0.186029,0.014724,0.054956,0.033903,0.178917,0.138213,6.459360e-01,0.518426,0.110012,NaN
103,314,0.436937,0.745451,0.800892,0.016995,0.533047,0.536181,0.034621,0.032831,0.183405,0.010376,0.052416,0.000494,0.409370,0.075634,4.565487e-02,0.465334,0.019495,NaN
104,315,0.378378,0.698736,0.740545,0.109796,0.402225,0.403611,0.045163,0.033364,0.183133,0.009598,0.053319,0.002755,1.000000,0.127320,2.051479e-01,0.537789,0.033813,NaN
105,316,0.175676,0.842218,0.889649,0.048520,0.424285,0.437246,0.024283,0.037098,0.180765,0.002493,0.059906,0.000886,0.299854,0.267862,3.628698e-02,0.213616,0.036920,NaN
106,317,0.396396,0.538413,0.558296,0.274828,0.387129,0.358865,0.096257,0.125709,0.309892,0.088493,0.109597,0.026798,0.478770,0.125849,5.343469e-01,0.547158,0.031890,NaN
107,318,0.261261,0.868658,0.914778,0.051716,0.796890,0.779040,0.054871,0.054044,0.184286,0.012115,0.081591,0.002596,0.684920,0.179798,7.862374e-02,0.434104,0.046695,NaN
108,319,0.036036,0.847942,0.881225,0.134791,0.429560,0.395658,0.103346,0.008278,0.181535,0.001143,0.019136,0.002672,0.478770,0.447887,7.871606e-02,0.101187,0.158553,NaN
109,320,0.252252,0.493964,0.545010,0.090701,0.327266,0.350823,0.011958,0.116244,0.318959,0.107328,0.091137,0.001325,0.380674,0.112443,3.348304e-02,0.367895,0.039763,NaN


In [19]:
# train_train['Y'] = train_train['Y'].apply(
#     lambda x: ((x - train_train['Y'].min())/(train_train['Y'].max()-train_train['Y'].min()))
# )
# train_val['Y'] = train_val['Y'].apply(
#     lambda x: ((x - train_val['Y'].min()) / (train_val['Y'].max() - train_val['Y'].min()))
# )
print("model training")
#模型训练
lgbmodel = lgb.LGBMRegressor(
    boosting_type='gbdt',
    objective='regression',
    num_leaves=127,
    max_depth=8,
    n_estimators=20000,
    learning_rate=0.05,
    # n_jobs=20,
    random_state=2018
)
lgbmodel.fit(
    X=train_train[use_feature_list],
    y=train_train['Y'],
    eval_set=(train_val[use_feature_list], train_val['Y']),
    early_stopping_rounds=500,
    verbose=True
)

fea_imp = pd.Series(lgbmodel.feature_importances_,use_feature_list).sort_values(ascending=False)
print(fea_imp)

model training
[1]	valid_0's l2: 0.00040212
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's l2: 0.000392492
[3]	valid_0's l2: 0.00038942
[4]	valid_0's l2: 0.000398831
[5]	valid_0's l2: 0.000391797
[6]	valid_0's l2: 0.000393866
[7]	valid_0's l2: 0.00039281
[8]	valid_0's l2: 0.000403057
[9]	valid_0's l2: 0.000397496
[10]	valid_0's l2: 0.000397049
[11]	valid_0's l2: 0.000388731
[12]	valid_0's l2: 0.000409004
[13]	valid_0's l2: 0.000419689
[14]	valid_0's l2: 0.000427807
[15]	valid_0's l2: 0.000420645
[16]	valid_0's l2: 0.000416354
[17]	valid_0's l2: 0.000415615
[18]	valid_0's l2: 0.000425029
[19]	valid_0's l2: 0.00043426
[20]	valid_0's l2: 0.000422992
[21]	valid_0's l2: 0.000422259
[22]	valid_0's l2: 0.000431666
[23]	valid_0's l2: 0.000428008
[24]	valid_0's l2: 0.000422255
[25]	valid_0's l2: 0.000431252
[26]	valid_0's l2: 0.000440182
[27]	valid_0's l2: 0.000439062
[28]	valid_0's l2: 0.000448594
[29]	valid_0's l2: 0.000437465
[30]	valid_0's l2: 0.000439042
[31]	

[273]	valid_0's l2: 0.0011834
[274]	valid_0's l2: 0.0011894
[275]	valid_0's l2: 0.00119779
[276]	valid_0's l2: 0.0011933
[277]	valid_0's l2: 0.00118856
[278]	valid_0's l2: 0.00118596
[279]	valid_0's l2: 0.00118794
[280]	valid_0's l2: 0.00118698
[281]	valid_0's l2: 0.00118287
[282]	valid_0's l2: 0.00118044
[283]	valid_0's l2: 0.00119217
[284]	valid_0's l2: 0.00120905
[285]	valid_0's l2: 0.00120813
[286]	valid_0's l2: 0.00121652
[287]	valid_0's l2: 0.00121171
[288]	valid_0's l2: 0.00121179
[289]	valid_0's l2: 0.00120651
[290]	valid_0's l2: 0.00120564
[291]	valid_0's l2: 0.00120798
[292]	valid_0's l2: 0.00121658
[293]	valid_0's l2: 0.00122212
[294]	valid_0's l2: 0.00122825
[295]	valid_0's l2: 0.00122391
[296]	valid_0's l2: 0.00122128
[297]	valid_0's l2: 0.00123413
[298]	valid_0's l2: 0.00122966
[299]	valid_0's l2: 0.0012252
[300]	valid_0's l2: 0.00123141
[301]	valid_0's l2: 0.00123047
[302]	valid_0's l2: 0.00123876
[303]	valid_0's l2: 0.00123389
[304]	valid_0's l2: 0.00123411
[305]	valid_

In [ ]:
data.drop_duplicates(subset=['TERMINALNO'],inplace=True)
train_data.drop_duplicates(subset=['TERMINALNO'],inplace=True)
test_data.drop_duplicates(subset=['TERMINALNO'],inplace=True)
data

In [ ]:
# 切割训练集和测试集
train = data[0:len(train_data)]
test = data[len(train_data):]

In [ ]:
train = pd.merge(train, feature, how='left', on='TERMINALNO')
test = pd.merge(test, feature, how='left', on='TERMINALNO')
train['Y'] = train['Y'].apply(lambda x: ((x-train['Y'].min())/(train['Y'].max()-train['Y'].min())))
# 训练集和验证集划分
train_train, train_val = train_test_split(train, test_size=0.2, random_state=42)
print("train_train_shape:"+str(train_train.shape)+"  train_val_shape:"+str(train_val.shape))

In [ ]:
train_train

In [ ]:
# train_train['Y'] = train_train['Y'].apply(
#     lambda x: ((x - train_train['Y'].min())/(train_train['Y'].max()-train_train['Y'].min()))
# )
# train_val['Y'] = train_val['Y'].apply(
#     lambda x: ((x - train_val['Y'].min()) / (train_val['Y'].max() - train_val['Y'].min()))
# )

#模型训练
lgbmodel = lgb.LGBMRegressor(num_leaves=31, max_depth=-1, n_estimators=20000,learning_rate=0.01,n_jobs=20,random_state=2018)
lgbmodel.fit(X=train_train[use_feature_list], y=train_train['Y'],
             eval_set=(train_val[use_feature_list], train_val['Y']), early_stopping_rounds=2000,verbose=False)

fea_imp = pd.Series(lgbmodel.feature_importances_,use_feature_list).sort_values(ascending=False)
print(fea_imp)

In [ ]:
train_val['pred']=lgbmodel.predict(train_val[use_feature_list])
train_val[['Y','pred']]